# A. BUILD FIRST MODEL
## (no data preparation)

### Install packages / import namespaces

In [1]:
#r "nuget:Microsoft.ML,1.5.0-preview2"
using Microsoft.ML;
using Microsoft.ML.Data;

Installed package Microsoft.ML version 1.5.0-preview2

### Load data models from Models.cs file 

In [2]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.cs"

### Initialize the ML context (we need it for building the data and training pipelines)  

In [3]:
MLContext mlContext = new MLContext(0);

### Load data from csv file into a dataview

In [4]:
const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

### Shuffle and split data to _trainingData_ and _testingData_ by a fraction of 0.2

In [5]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 0);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

### Model builder pipeline

In [6]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared" };

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

### Train the model

In [7]:
var model = trainingPipeline.Fit(trainingData);

In [8]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

In [9]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;

Installed package Microsoft.Data.Analysis version 0.4.0

#### Load formatter for metrics

In [10]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.cs"
var categories = new string[] { "FlashLight", "Infrared", "Day", "Lighter" };
Formatters.Load(categories);

In [11]:
metrics

EVALUATION: multi-class classification Class Value Note MacroAccuracy 0.986 the closer to 1, the better MicroAccuracy 0.988 the closer to 1, the better LogLoss 0.090 the closer to 0, the better LogLoss per Class FlashLight 0.074 the closer to 0, the better Infrared 0.140 the closer to 0, the better Day 0.034 the closer to 0, the better Lighter 0.102 the closer to 0, the better

In [12]:
var sampleData = new ModelInput
{
    Luminosity = 92F,
    Temperature = 122F,
    Infrared = 0F
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[style: "font-size: 24px"]($"{predicted.PredictedLabel}"));

FlashLight

### Save the model

In [13]:
mlContext.Model.Save(model, trainingData.Schema, "model.zip");

In [14]:
#r "nuget:Microsoft.ML.OnnxConverter,0.17.0-preview2"

Installed package Microsoft.ML.OnnxConverter version 0.17.0-preview2

### Save the model in ONNX format 

In [15]:
using (var stream = System.IO.File.Create("model.onnx"))
{
    mlContext.Model.ConvertToOnnx(model, data, stream);
}